Ce notebook est un premier pas avant transformation en application.
Il nous permet de tester le (faible) niveau d'accessibilité de ce format notebook pour les personnes aveugles.

Quelles règles apprises : 
- Le notebook n'est globalement pas très accessible
- On peut quand même améliorer les choses en choisissant une langue source (pour faciliter le travail de l'aide audio de la personne aveugle) et en mettant une structure en titres/sous-titre facilitant la navigation.

## Preprocessing des données

In [ ]:
import pandas as pd

Hyperparamètres

In [ ]:
cols_access = ['entree_balise_sonore'] # Variable d'accessibilité visée
# code_com_cible = "85294" # Commune d'intérêt
code_com_cible = "93066"

Download data acceslibre

In [ ]:
url_access_libre = "https://www.data.gouv.fr/fr/datasets/r/5b0f44f2-e6ea-4a58-874d-6fe364b40342"
df_accesslibre = pd.read_csv(url_access_libre)

In [ ]:
df_accesslibre.columns

Columns selection

In [ ]:
cols_base = ['id', 'name', 'code_insee', 'postal_code', 'commune', 'numero', 'voie', 'lieu_dit', 'longitude', 'latitude', ]
df_balises = df_accesslibre[cols_base + cols_access]


Suppression des établissements sans information sur les balises sonores

In [ ]:
df_balises = df_balises[df_balises['entree_balise_sonore'].notna()]
print(df_balises.shape)

In [ ]:
df_balises['entree_balise_sonore'].value_counts()

In [ ]:
df_balises

Cleaning des positions GPS

In [ ]:
# Convertir les colonnes latitude et longitude si nécessaire
df_balises['longitude'] = pd.to_numeric(df_balises['longitude'], errors='coerce')
df_balises['latitude'] = pd.to_numeric(df_balises['latitude'], errors='coerce')

# Supprimer les lignes avec des coordonnées invalides
df_balises = df_balises.dropna(subset=['longitude', 'latitude'])


On ne garde que les établissements d'une communes. 

In [ ]:
df_balises_com = df_balises[df_balises["code_insee"]==code_com_cible]


## Résultats

### Tableau des établissements répertoriés dans Accès Libre à La Tranche sur mer

In [ ]:
df_balises_com[["name","entree_balise_sonore","numero","voie", "commune"]]

In [ ]:
nb_etab_equipes = df_balises_com["entree_balise_sonore"].sum()
if (nb_etab_equipes>0):
    print(f"{nb_etab_equipes} établissements sont équipés dans la communes")
else:
    print("Aucun d'établissement équipé dans la commune")

### Carte des établissements

In [ ]:
import folium

point_central = [df_balises_com['latitude'].mean(), df_balises_com['longitude'].mean()]
carte = folium.Map(location=point_central, zoom_start=6)

coordonnees = []
for _, row in df_balises_com.iterrows():
    if row['entree_balise_sonore'] == False:
        continue
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"<b>{row['name']}</b>",
    ).add_to(carte)
    coordonnees.append([row['latitude'], row['longitude']])

if coordonnees:
    carte.fit_bounds(coordonnees)
display(carte)


In [ ]:
from gtts import gTTS
import os
from playsound import playsound

if nb_etab_equipes == 0:
    text = "Aucune balise sonore n'est installée dans la zone ciblée."
else:
    text = f"Il y a {nb_etab_equipes} balises sonores dans la zone ciblée. Voici les noms des établissements équipés : "
    for i in range(nb_etab_equipes):
        text += f"{df_balises_com.iloc[i]['name']}."

print(text)

# Text to speech
tts = gTTS(text=text, lang='fr')

# Save play, delete
audio_file = "97r08weurdshsudczkcb;kjxznckjxzc.mp3"
tts.save(audio_file)
playsound(audio_file)
os.remove(audio_file)
